<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/mntrf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
excllcode = data.iloc[:, 2]
excllcode

0      9000232
1      9000040
2      1001452
3      9000041
4      9000239
        ...   
182    2018455
183    2018445
184    9000005
185    9000006
186    9000007
Name: excllncobsrvtcode, Length: 187, dtype: int64

In [14]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntrf/list'
key = 'jkd4PnVjssNP+g2WeRzs8lgc4SOelLsncvuMr81/9R9nSYKkiQAcb2xt8GTYgTJSQGQ4+htuRuyIXSwRXr210w=='
 
api_key = requests.utils.unquote(key) #디코딩

# 우량 관측소 코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/excllcode.csv')

# 우량 관측소 코드 조회
excllcode = data.iloc[:, 2]
xmlrows = []
for code in excllcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                quote_plus('sdate') : "2020-01-02",
                                quote_plus('stime') : "00",
                                quote_plus('edate') : "2020-01-02",
                                quote_plus('etime') : "24",
                                quote_plus('excll') : code, #우량관측소 코드
                                quote_plus('tms') : "10", #10분 단위
                                quote_plus('numOfRows') : "800000",
                                quote_plus('pageNo') : "1"})
  
#  print(Params)
  
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)

print (xmlrows)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultMsg></header></response>'
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response></body></html>
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultMsg></header></response>'
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>99</resultcode><resultmsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</resultmsg></header></response></body></html>
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.</result

In [16]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'


with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [20]:
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break #반복문 종료
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + nameList


IndexError: ignored

In [25]:
xmlrows[0]

[]

In [18]:
total = []
excllcodeLen = len(excllcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [excllcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, excllcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

   

In [19]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/mntrf_20.01.02_2.csv",encoding="utf-8-sig")

In [ ]:
result